# 술 추천 알고리즘

도수 유사도는 `euclidean distance`로 구했고, 풍미 및 기타 정보의 유사도는 `cosine similarity`로 구했다.  
술의 고유 id를 받아 비슷한 술의 고유 id를 return한다.

### general_recommendation():
도수 유사도 50% 풍미(6가지) 유사도 50% 고려해 주종 상관 없이 비슷한 술을 찾아줌

### wine_recommendation():
도수 유사도 50% 기타(category, origin, producer, wine_grape, flavour) 50% 고려해 같은 대분류 내에서 비슷한 와인을 찾아줌

### the used variables of each alcohols
(현재는 wine recommendation만 구축됨! makgeoli, beer, vodka, soju, whisky, korean도 variable만 바꾸어주어 만들면 됨.)  

all: percent, flavour  
wine: category, percent, **origin**, **producer**, **wine_grape**, flavour  
makgeoli: category, percent, **producer**, flavour  
beer: category, percent, **origin**, **producer**, flavour  
vodka: **origin**, producer, flavour  
soju: category, percent, **producer**, flavour  
whisky: category, **whisky_category**, percent, **origin**, flavour  
korean: category, percent, flavour  

### the explanation of variables
id: 술의 고유 id  
class: 주종  
category: 중분류  
name: 이름  
percent: 도수  
origin: 원산지/생산지  
producer: 제조사/생산자  
wine_grape: 포도 품종 (와인만)  
whisky_category: 위스키 소분류 (위스키만)  
sweet: 달콤한  
light: 가벼운  
soft: 부드러운  
bitter: 쓴맛이 강한  
clean: 깔끔한  
smell: 향이 강한  

In [1]:
# 여러 주종 내 추천
# 주종에 상관 없이 도수와 풍미만 고려해 비슷한 술을 추천해줌
# 항목: 도수, 풍미 (약 6개 항목)

import numpy as np
import pandas as pd 
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from scipy.spatial import distance_matrix
from scipy.spatial.distance import squareform
from sklearn.preprocessing import MinMaxScaler

# data = pd.read_csv("data/drinksData.csv")

# 확인
# data.head()

In [2]:
import pymysql

# MySQL Connection
conn = pymysql.connect(host='localhost', user='root', password='1234', db='sooljari', charset='utf8')
curs = conn.cursor()

sql = "select * from product"
curs.execute(sql)

rows = curs.fetchall()

colname = curs.description

col = []

for i in colname :
    col.append(i[0])
    
emp = pd.DataFrame(list(rows), columns=col)
data = emp[['id','area','description','name','price','proof','image','category','sweet','light','soft','bitter','clean','smell']]
data

,id,area,description,name,price,proof,image,category,sweet,light,soft,bitter,clean,smell
0,1,양주,#팡팡 터지는 화사한 생강 향,도봉산 막걸리 8% X 2병,"18,000",8.0,https://www.sooldamhwa.com/_next/image?url=htt...,탁주,1,1,0,0,0,1
1,2,김천,#초코라테 색감을 가진 새콤달콤한 포도 막걸리,배금도가 포도막걸리,"15,000",12.0,https://www.sooldamhwa.com/_next/image?url=htt...,탁주,1,1,0,0,1,1
2,3,서울,#시원한 참외 향에 청량감은 up!,마크홀리 오리지널 6.0,"23,700",6.0,https://www.sooldamhwa.com/_next/image?url=htt...,탁주,1,1,1,0,0,1
3,4,나주,"#상큼함이 따봉, 짜릿한 탄산도 따봉",참주가 라봉 막걸리 X,"24,000",5.5,https://www.sooldamhwa.com/_next/image?url=htt...,탁주,1,1,1,0,0,1
4,5,포천,#달콤한 맛과 부드러운 질감,담은 블랙,"45,000",6.5,https://www.sooldamhwa.com/_next/image?url=htt...,탁주,1,1,1,0,0,1
5,6,오산,#달지 않은 담백함,오산 막걸리,"12,000",6.0,https://www.sooldamhwa.com/_next/image?url=htt...,탁주,1,1,0,0,1,1
6,7,김천,#달콤쌉쌀 포도약주,배금도가 포도약주,"22,500",16.0,https://www.sooldamhwa.com/_next/image?url=htt...,약·청주,0,0,1,1,1,1
7,8,충주,#구수한 쌀의 맛,동학 1957,"3,500",13.0,https://www.sooldamhwa.com/_next/image?url=htt...,약·청주,0,1,1,0,1,1
8,9,나주,#2200시간의 기다림,나주 배 약주,"20,000",15.0,https://www.sooldamhwa.com/_next/image?url=htt...,약·청주,0,0,0,1,1,1
9,10,경남,#고소한 쌀 튀밥 향,청비성,"24,000",16.0,https://www.sooldamhwa.com/_next/image?url=htt...,약·청주,0,0,1,1,1,1


In [3]:
# 도수 유사도 구하기
percent = data['proof']
dist_pair = []

# y축에 임의로 0을 부여한 거리 순서쌍 생성
for i in range(0,len(percent)):
    temp = []
    temp.append(data.loc[i]['proof'])
    temp.append(0)
    dist_pair.append(temp)

# get a distance matrix
df = pd.DataFrame(dist_pair, columns=['x', 'y'])
dist_matrix = distance_matrix(df.values, df.values)

# 정규화
min_max_scaler = MinMaxScaler()
regularised = min_max_scaler.fit_transform(dist_matrix)

# 1에서 빼줘서 더 가까운 것이 우선순위를 갖도록 변경하기
# 데이터 숫자만큼으로 변경해줘야 함
one_matrix = np.ones((25,25))

final_dist = one_matrix - regularised

# 확인
print(final_dist)

[[1.         0.87096774 0.94594595 0.93333333 0.95890411 0.94594595
  0.7037037  0.83333333 0.75       0.7037037  0.75       0.75
  0.92105263 0.97058824 0.91863517 0.88888889 1.         0.87096774
  0.08587258 0.08136483 0.08831909 0.54166667 0.15422886 0.11439114
  0.83333333]
 [0.88571429 1.         0.83783784 0.82666667 0.84931507 0.83783784
  0.85185185 0.96666667 0.89285714 0.85185185 0.89285714 0.89285714
  0.81578947 0.91176471 0.81364829 0.98412698 0.88571429 1.
  0.1966759  0.18635171 0.2022792  0.70833333 0.35323383 0.26199262
  0.96666667]
 [0.94285714 0.80645161 1.         0.98666667 0.98630137 1.
  0.62962963 0.76666667 0.67857143 0.62962963 0.67857143 0.67857143
  0.97368421 0.91176471 0.97112861 0.82539683 0.94285714 0.80645161
  0.03047091 0.02887139 0.03133903 0.45833333 0.05472637 0.04059041
  0.76666667]
 [0.92857143 0.79032258 0.98648649 1.         0.97260274 0.98648649
  0.61111111 0.75       0.66071429 0.61111111 0.66071429 0.66071429
  0.98684211 0.89705882 0.98

In [4]:
# flavour word list 만들기
flavour_list = [] # empty list

for i in range(0, len(data)):
    temp = ""
    
    if data.loc[i]['sweet'] == 1:
        temp = temp + "sweet "
    if data.loc[i]['light'] == 1:
        temp = temp + "light "
    if data.loc[i]['soft'] == 1:
        temp = temp + "soft "
    if data.loc[i]['bitter'] == 1:
        temp = temp + "bitter "
    if data.loc[i]['clean'] == 1:
        temp = temp + "clean "
    if data.loc[i]['smell'] == 1:
        temp = temp + "smell "
    
    flavour_list.append(temp)

# 해당 리스트 데이터 프레임에 추가
data["flavour"] = pd.DataFrame({"flavour":flavour_list})
flavour = data['flavour']

# flavour 코사인 유사도 구하기
# instantiating and generating the count matrix
count = CountVectorizer()
count_matrix = count.fit_transform(flavour)

# generating the cosine similarity matrix
cosine_sim = cosine_similarity(count_matrix, count_matrix)

# 확인
print(cosine_sim)

[[1.         0.8660254  0.8660254  0.8660254  0.8660254  0.8660254
  0.28867513 0.57735027 0.33333333 0.28867513 0.28867513 0.33333333
  0.77459667 0.8660254  0.8660254  0.8660254  0.77459667 0.8660254
  0.         0.         0.         0.40824829 0.         0.
  1.        ]
 [0.8660254  1.         0.75       0.75       0.75       1.
  0.5        0.75       0.57735027 0.5        0.5        0.28867513
  0.89442719 1.         1.         0.75       0.89442719 1.
  0.28867513 0.28867513 0.         0.35355339 0.35355339 0.35355339
  0.8660254 ]
 [0.8660254  0.75       1.         1.         1.         0.75
  0.5        0.75       0.28867513 0.5        0.5        0.57735027
  0.89442719 0.75       0.75       1.         0.89442719 0.75
  0.28867513 0.28867513 0.35355339 0.35355339 0.         0.
  0.8660254 ]
 [0.8660254  0.75       1.         1.         1.         0.75
  0.5        0.75       0.28867513 0.5        0.5        0.57735027
  0.89442719 0.75       0.75       1.         0.89442719 0

C:\Users\82106\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [5]:
# 도수, flavour를 모두 고려한 similarity 구하기 (weight는 각각 0.5)
new_sim = 0.5 * cosine_sim + 0.5 * final_dist

print(new_sim)

[[1.         0.86849657 0.90598567 0.89967937 0.91246476 0.90598567
  0.49618942 0.7053418  0.54166667 0.49618942 0.51933757 0.54166667
  0.84782465 0.91830682 0.89233029 0.87745715 0.88729833 0.86849657
  0.04293629 0.04068241 0.04415954 0.47495748 0.07711443 0.05719557
  0.91666667]
 [0.87586984 1.         0.79391892 0.78833333 0.79965753 0.91891892
  0.67592593 0.85833333 0.73510371 0.67592593 0.69642857 0.59076614
  0.85510833 0.95588235 0.90682415 0.86706349 0.89007074 1.
  0.24267552 0.23751342 0.1011396  0.53094336 0.35339361 0.30777301
  0.91634604]
 [0.90444127 0.77822581 1.         0.99333333 0.99315068 0.875
  0.56481481 0.75833333 0.48362328 0.56481481 0.58928571 0.62796085
  0.9340557  0.83088235 0.8605643  0.91269841 0.91864217 0.77822581
  0.15957302 0.15877326 0.19244621 0.40594336 0.02736318 0.0202952
  0.81634604]
 [0.89729842 0.77016129 0.99324324 1.         0.98630137 0.86824324
  0.55555556 0.75       0.47469471 0.55555556 0.58035714 0.61903228
  0.94063465 0.82352

In [6]:
# 여러 주종 내 추천
# 주종에 상관 없이 도수와 풍미만 고려해 비슷한 술을 추천해줌
# 항목: 도수, 풍미 (약 6개 항목)

# 고유 id를 넣으면 해당 술과 비슷한 Top 10의 id를 return
def general_recommendation(input_id, new_sim = new_sim):
    
    # 고유 id로 index 찾기 
    idx = data.index[data['id'] == input_id].tolist() # Int64Index 형식이라 list로 바꾸어줌
    
    # 해당 index의 유사도 리스트 sort in descending order
    score_series = pd.Series(new_sim[idx[0]]).sort_values(ascending = False)
    
    # 유사도 Top 10의 index 추출
    top_10_indexes = list(score_series.iloc[1:11].index)

    # 유사도 1인 항목이 하나 더 있어서 자기 자신이 포함되는 경우에는 자신을 뺀 Top 10의 index 재추출
    if top_10_indexes[0] == idx[0]:
        top_10_indexes = list(score_series.iloc[1:12].index)
        top_10_indexes.remove(idx[0])
    
    # 고유 id를 담기 위한 empty list 생성
    top_10_id = []
    
    # id list
    for i in top_10_indexes:
        id = data.loc[i]['id']
        top_10_id.append(id)
    
    return top_10_id

# test
print(general_recommendation(10))

[11, 9, 12, 8, 17, 13, 18, 2, 16, 14]


In [7]:
# test
# 테스트로 임의의 아이디 넣음
test_num = 2

result = (general_recommendation(test_num))
print(result)

# print(is_wine(test_num))

# print(wine_recommendation(test_num))

[14, 6, 25, 15, 17, 1, 16, 8, 13, 5]


In [8]:
type(result)

list

In [9]:
df = data.loc[data['id'].isin(result)]
df2 = df.reset_index(drop=True)
df3 = df2.transpose()
df3.head()

,0,1,2,3,4,5,6,7,8,9
id,1,5,6,8,13,14,15,16,17,25
area,양주,포천,오산,충주,영동,세종,충주,영천,양평,서울
description,#팡팡 터지는 화사한 생강 향,#달콤한 맛과 부드러운 질감,#달지 않은 담백함,#구수한 쌀의 맛,#음료수처럼 새콤달콤 맛있어!,#복숭아 잼처럼 진득한 단맛,#사과와 배의 만남 파티주로 강추!,#자두로 만든 달콤한 와인,#아카시아꿀로 만든 술,테스트
name,도봉산 막걸리 8% X 2병,담은 블랙,오산 막걸리,동학 1957,피에스 화이트 스파클링 와인,오롯이 복숭아 스파클링 와인,허니문배,고도리 자두 와인,허니비 와인,test
price,"18,000","45,000","12,000","3,500","38,000","15,000","17,900","25,000","35,000",10000


In [12]:
js = df3.to_json(force_ascii=False)
print(js)

{"0":{"id":1,"area":"양주","description":"#팡팡 터지는 화사한 생강 향","name":"도봉산 막걸리 8% X 2병","price":"18,000","proof":8.0,"image":"https:\/\/www.sooldamhwa.com\/_next\/image?url=https%3A%2F%2Fd38cxpfv0ljg7q.cloudfront.net%2Fadmin_contents%2Fthumbnail%2Froh2-1660799941081-1.jpg&w=1080&q=75","category":"탁주","sweet":1,"light":1,"soft":0,"bitter":0,"clean":0,"smell":1,"flavour":"sweet light smell "},"1":{"id":5,"area":"포천","description":"#달콤한 맛과 부드러운 질감","name":"담은 블랙","price":"45,000","proof":6.5,"image":"https:\/\/www.sooldamhwa.com\/_next\/image?url=https%3A%2F%2Fd38cxpfv0ljg7q.cloudfront.net%2Fadmin_contents%2Fthumbnail%2FLNfl-1651807247828-1.jpg&w=1080&q=75","category":"탁주","sweet":1,"light":1,"soft":1,"bitter":0,"clean":0,"smell":1,"flavour":"sweet light soft smell "},"2":{"id":6,"area":"오산","description":"#달지 않은 담백함","name":"오산 막걸리","price":"12,000","proof":6.0,"image":"https:\/\/www.sooldamhwa.com\/_next\/image?url=https%3A%2F%2Fd38cxpfv0ljg7q.cloudfront.net%2Fadmin_contents%2Fthumbnail%2FHm

In [ ]:
from flask import Flask
from flask import jsonify
from flask_cors import CORS

app = Flask(__name__)
cors = CORS(app, resources = {r"/recommend/*": {"origins" : "*"}})

@app.route("/recommend", methods=['GET', 'POST'])
def test() :
    return js

if __name__ == "__main__" :
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [08/Nov/2022 12:30:30] "GET /recommend HTTP/1.1" 200 -
